# **Project Name**    -Unsupervised ML - Online Retail Customer Segmentation


##### **Project Type**    - Unsupervised ML - Online Retail Customer Segmentation
##### **Prepared by**     - Anurudra Jena



# **Project Summary -**

In this project, our task is to identify major customer segments on a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

The data collection was already completed and can be found in many different data/ML workshop websites. e.g.- Kaggle

Further tasks to be completed are as follows:
1. Preprocessing:
Clean and preprocess the data by handling missing values, removing outliers, and normalizing or scaling numerical features.

2. Feature Selection:
Selecting appropriate features for clustering that can effectively differentiate customers and capture meaningful patterns. For this project, a subset of relevant features such as customer demographics, purchase behavior, or interaction history will be used.

3. Generating RFM Score:
Just before doing any segmentation/clustering, its recommended that we do an EDA on the data to get a firm understanding of the data and its impact on the business.

4. EDA:
Creating RFM analysis allows us to compare between potential contributors or clients. It gives organizations a sense of how much revenue comes from repeat customers (versus new customers), and which levers they can pull to try to make customers happier so they become repeat purchasers.

5. Choosing a Clustering Algorithm:
Selecting a suitable clustering algorithm based on the nature of the data and project requirements. Commonly used algorithms for customer segmentation include K-means clustering, hierarchical clustering, DBSCAN, or Gaussian mixture models (GMM). But for this project we'll be using K-Means clustering.

6. Evaluation and Validation:
The quality of clustering results using appropriate metrics such as silhouette score, Davies-Bouldin index, or within-cluster sum of squares (WCSS) for K-means will be evaluated.
The clusters will be validated by assessing their coherence and interpretability in relation to business objectives and domain knowledge.

# **GitHub Link -**

Provide your GitHub Link here.

# ***Let's Begin !***

## ***1. Knowing the data***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from numpy import math
import datetime as dt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import metrics
from yellowbrick.cluster import KElbowVisualizer

### Dataset Loading

In [ ]:
df_raw = pd.read_csv('/content/sample_data/Online Retail.csv', encoding='unicode_escape')

### Dataset First View

In [ ]:
df_raw.head()

### Dataset Rows & Columns count

In [ ]:
df_raw.shape

### Dataset Information

In [ ]:
df_raw.info()

#### Duplicate Values

In [ ]:
df_raw.duplicated().sum()

#### Missing Values/Null Values

In [ ]:
df_raw.isnull().sum()

In [ ]:
sns.heatmap(df_raw.isna())

### What did you know about your dataset?

After looking at the dataset for missing and duplicated values, it can be observed that most of them are pretty negligible with respect to the size of the dataset, except for the CustomerID column which has about 40k missing values.

Although it can be said that since the number of missing CustomerID doesn't account for the missing customer sales that didn't happen and we can ofcourse get a greater and much more valid insight if instead of removing those rows, we can add trial customerIDs so that their purchases and inputs can be count for segmentation purpose. Although for the sake of gaining business insights we have to eliminate those customers.

## ***2. Understanding Your Variables***

In [ ]:
df_raw.select_dtypes(include = ['int64','float64']).columns

In [ ]:
df_raw.select_dtypes(include = ['object']).columns


### Variables Description

In [ ]:
df_raw.describe()

### Check Unique Values for each variable.

In [ ]:
df_raw.nunique()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# creating a boxplot to check the price distribution density and analyse the ouliers in the group

sns.boxplot(x = 'Quantity', data = df_raw)
plt.show()

# creating a boxplot to check the Minimum_nights distribution density and analyse the ouliers in the group

sns.boxplot(x = 'UnitPrice', data = df_raw)
plt.show()

In [ ]:
# quantifying the price variable in percentiles to check the outliers in the group

df_raw.loc[:,'UnitPrice'].quantile([x for x in np.arange(0.1,1.0,0.1)]+[0.98]+[0.99]+[0.995]+[0.999]+[0.9995]+[0.9999])

In [ ]:
df_raw.loc[:,'Quantity'].quantile([x for x in np.arange(0.1,1.0,0.1)]+[0.98]+[0.99]+[0.995]+[0.999]+[0.9995]+[0.9999])

In [ ]:
df_price_handled = df_raw[(df_raw['UnitPrice'] <= df_raw['UnitPrice'].quantile(0.9995))]

Dropping duplicates first !

In [ ]:
df_price_handled.drop_duplicates(inplace=True)

First removing the columns with 350+ null Description values.

In [ ]:
df_filtered = df_price_handled[df_price_handled['Description'].notna()]

Then we have a DF where only the CustomerID column will be having null values.

In [ ]:
def fill_empty_with_random(df):
    for col in df.columns:
        if df[col].isna().any():
            np.issubdtype(df[col].dtype, np.number)
            random_values = np.random.randint(20000, 99999, size=len(df))

            df[col].fillna(pd.Series(random_values), inplace=True)

    return df

df_filtered = fill_empty_with_random(df_filtered)

Lets check how much null values did our function filled out.

In [ ]:
df_filtered.isnull().sum()

As we caan see almost 3363 null values are still there in the CustomerID section maybe because of being different data type. This can be handled by adding more conditionals in the function or using error handling techniques.
But for the sake of simplicity, we can simply remove the left out null values which aren't a significant part of the entire dataset.

Now all the missing customerID values in the dataset has been filled out with garbage/raandom values which shall act individual customers and would weigh in while we do our segmentation.

In [ ]:
df_filtered.dropna(inplace = True)

Now that the null values of CustomerID has been handled, we can remove the null values form the Description column which is a very non-ssignificant part of the entire dataset.

In [ ]:
df_filtered.isnull().sum()

Now we don't have any more null values, so we can move towards changing required data types.


In [ ]:
df_filtered['InvoiceNo'] = df_filtered['InvoiceNo'].astype('str')

Removing cancelled orders to gain insights only on valid orders.

In [ ]:
df_filtered = df_filtered[~df_filtered['InvoiceNo'].str.contains('C')]

Since more than 40 orders have UnitPrice as zero, those can be removed considering those items are add-ons to some real items.

In [ ]:
df_f = df_filtered[df_filtered['UnitPrice']>0]
df_f.head()

In [ ]:
df_f.shape
df_f.describe()

### What all manipulations have you done and insights you found?

The outliers from the UnitPrice and Quantity column has been rermoved and the missing values from the CustomerID and Description has been handled.

In addition to that, the cancelled orders and the orders with zero UnitPrice has also been removed.

## ***4. Feature Engineering & Data Pre-processing***

Converting Invoice date to Date-time format

In [ ]:
df_f['InvoiceDate'] = pd.to_datetime(df_f['InvoiceDate'], format='mixed')

In [ ]:
df_f['day'] = df_f['InvoiceDate'].dt.day_name()
df_f['year'] = df_f['InvoiceDate'].apply(lambda x: x.year)
df_f['month_num'] = df_f['InvoiceDate'].apply(lambda x: x.month)
df_f['day_num'] = df_f['InvoiceDate'].apply(lambda x: x.day)
df_f['hour'] = df_f['InvoiceDate'].apply(lambda x: x.hour)
df_f['minute'] = df_f['InvoiceDate'].apply(lambda x: x.minute)
df_f['month'] = df_f['InvoiceDate'].dt.month_name()

## ***5.*** ***EDA***

**Top 10 Highest selling products**

In [ ]:
top10_products = df_f['Description'].value_counts().reset_index().head(10)
print(top10_products)

Plotting the above data to get a perspective.

In [ ]:
plt.figure(figsize = (15,8))
sns.barplot(x= top10_products['count'], y = top10_products['Description'])

**Top 10 spending Customers**

In [ ]:
top10_customers = df_f['CustomerID'].value_counts().reset_index().head(10)
print(top10_customers)

Remember, we had put some random customerID values... ??

The range for Random values were from 20000 to 99999, and as we can see none of the top 10 customers are having customer_ID in that range, we can say that our drive to save the data didn't hamper any insights.

Now lets plot the above data and get some perspective. We can use Bar chart as the data is quite normalized.

In [ ]:
plt.figure(figsize = (15,8))
sns.barplot(x= top10_customers['CustomerID'], y = top10_customers['count'])

**Top 5 buying countries**

In [ ]:
top5_countries = df_f['Country'].value_counts().reset_index().head(5)
print(top5_countries)

In [ ]:
plt.figure(figsize=(8, 6))

colors = sns.color_palette('pastel')
plt.pie(top5_countries['count'], labels=top5_countries['Country'], colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Country retail share')
plt.show()

**Monthly retail sales**

In [ ]:
monthly_sales = df_f['month'].value_counts().reset_index()
print(monthly_sales)

Plotting the above data:

In [ ]:
plt.figure(figsize = (15,8))
sns.barplot(x= monthly_sales['month'], y = monthly_sales['count'])
plt.title("Sales count on different months")

**Day-to-Day sales**

In [ ]:
daily_sales = df_f['day'].value_counts().reset_index()
print(daily_sales)

Plotting the abpve data for better insights.

In [ ]:
plt.figure(figsize = (15,8))
sns.barplot(x= daily_sales['day'], y = daily_sales['count'])
plt.title("Sales count on different days")

**Time of the day sales figures**

In [ ]:
df_f['hour'].unique()

In [ ]:
def day_time(time):
  if time>=6 and time<=11:
    return "Morning"
  elif time>=12 and time<= 17:
    return "Afternoon"
  else: return "Evening"

df_f['DayTime'] = df_f['hour'].apply(day_time)

In [ ]:
sales_time = df_f['DayTime'].value_counts().reset_index()
print(sales_time)

Plotting the above data for better insight:

In [ ]:
plt.figure(figsize=(8, 6))

colors = sns.color_palette('pastel')
plt.pie(sales_time['count'], labels=sales_time['DayTime'], colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Day time Sales')
plt.show()

**Average amount spent by each customer**

In [ ]:
df_f['TotalAmount'] = df_f['Quantity']*df_f['UnitPrice']

In [ ]:
avg_amt = df_f.groupby('CustomerID')['TotalAmount'].mean().reset_index().rename(columns = {'TotalAmount':'Avg_amt_per_customer'}).sort_values('Avg_amt_per_customer', ascending = False).head(10)
avg_amt

Plotting the top 10 values of the above data:

In [ ]:
plt.figure(figsize = (15,8))
sns.barplot(x= avg_amt['CustomerID'], y = avg_amt['Avg_amt_per_customer'])
plt.title("Average amount spent by the top 10 customers")

**Observations**

The top 5 selling products all yearr around:
1.  WHITE HANGING HEART T-LIGHT HOLDER    282
2.            REGENCY CAKESTAND 3 TIER    211
3.         HAND WARMER BABUSHKA DESIGN    185
4.        SCOTTIE DOG HOT WATER BOTTLE    177
5.               HEART OF WICKER SMALL    176

The top 5 most spending customers:
1.     12748.0    666
2.     17841.0    387
3.     14606.0    321
4.     14911.0    304
5.     17850.0    297

Top 5 buying Countries:

1.  United Kingdom  46045
2.         Germany    760
3.          France    650
4.            EIRE    386
5.           Spain    266

Most of the sales happen during the fall and within the Afternoon followed by Morning time.

The sales rate is increases as the weekend approaches, the highet happens on Fridays and the lowest happens on Sundays, deriving the fact that people happen to gift mostly on weekends and make last minute purchases. Customized offers can be made to rescue the sales figure on Sundays.

## ***6. RFM Scoring***

RFM analysis numerically ranks a customer in each of these three categories, generally on a scale of 1 to 5 (the higher the number, the better the result). The "best" customer would receive a top score in every category.
These three RFM factors can be used to reasonably predict how likely (or unlikely) it is that a customer will do business again with a firm or, in the case of a charitable organization, make another donation.

The RFM model is based on three quantitative factors:
* Recency: How recently a customer has made a purchase
* Frequency: How often a customer makes a purchase
* Monetary Value: How much money a customer spends on purchases

Since UK holds the significant share of sales, we can only focus on the UK Customers

In [ ]:
df_f.shape

In [ ]:
df_uk = df_f[df_f['Country'] == 'United Kingdom']

# print the shape of the data
df_uk.shape

In [ ]:
df_uk.describe()

In [ ]:
# To calculate the RFM, we need to select a date from which we can calculate the RFM, and since it is an old dataset, we can select a date with rrespect to the max date of the data set.

latest_date = dt.datetime(2011,1,11)

df_uk = df_uk.groupby('CustomerID').agg({'InvoiceDate': lambda x: (latest_date - x.max()).days, 'InvoiceNo': lambda x: len(x),
                                            'TotalAmount': lambda x: x.sum()})
df_uk['InvoiceDate'] = df_uk['InvoiceDate'].astype(int)

# rename columns to frequency, recency, monetary
df_uk.rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalAmount': 'Monetary'}, inplace=True)


In [ ]:
df_uk.shape

Plotting the RFM distribution:

In [ ]:
count = 1
plt.subplots(figsize=(20,13))
for feature in df_uk:
  plt.subplot(2,2,count)
  sns.distplot(df_uk[feature])
  plt.title(f"Distribution of the variable {feature}", fontsize=16)
  plt.xlabel(f"{feature}")
  plt.ylabel("Density")
  count += 1

We see that the distribution is uneven, and skewed for Recency and Frequency.
To normalize the curves, we can apply Log Transformation to Normalize the data.

In [ ]:
# since we do not see any negative values in the RFM scorre dataset, but we can ssee zerro value, we can convert all zero values to 1
# and directly apply log Transformation to the RFM values

def handle_negative(num):
  if num <= 0:
    return 1
  else:
    return num

# apply the function to recency and monetary columns
df_uk['Recency'] = [handle_negative(x) for x in df_uk['Recency']]
df_uk['Monetary'] = [handle_negative(x) for x in df_uk['Monetary']]
log_df = df_uk[['Recency', 'Frequency', 'Monetary']].apply(np.log, axis=1).round(3)

In [ ]:
count = 1
plt.subplots(figsize=(20,13))
for feature in log_df:
  plt.subplot(2,2,count)
  sns.distplot(log_df[feature])
  plt.title(f"Distribution of the variable {feature}", fontsize=16)
  plt.xlabel(f"{feature}")
  plt.ylabel("Density")
  count += 1

In [ ]:
#Applying log transformation to the original RFM values

df_uk['Recency_log'] = df_uk['Recency'].apply(math.log)
df_uk['Frequency_log'] = df_uk['Frequency'].apply(math.log)
df_uk['Monetary_log'] = df_uk['Monetary'].apply(math.log)

In [ ]:
#Scaling the data:
X_features = df_uk[['Recency_log', 'Frequency_log', 'Monetary_log']].values
scaler = StandardScaler()
X = scaler.fit_transform(X_features)

## **7. Implementing K-Means algorithm for un-supervised learning**

Before clustering, we have to inspect the RFM Value and apply Elbow/Silhoutte method to determine the number of clusters we want the Customers to be divided into.

For this case we'll be applying Elbow method:

In [ ]:
# Applying Elbow method

SSE = {}
for k in range(1,15):
  km = KMeans(n_clusters = k, init = 'k-means++', max_iter = 1000)
  km = km.fit(X)
  SSE[k] = km.inertia_

# plot the graph for SSE and number of clusters
visualizer = KElbowVisualizer(km, k=(1,15), metric='distortion', timings=False)
visualizer.fit(X)
visualizer.poof()
plt.show()


From the Elbow method, we can see that the optimal number of clusters is 4.

In [ ]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_km = kmeans.predict(X)

# Plot the clusters
plt.figure(figsize=(8, 5))
plt.title('Customer Segmentation based on Recency and Frequency')
plt.scatter(X[:,0], X[:,1], c=y_km, s=50, cmap='Set1', label='Clusters')

# Plot and annotate the centers
centers = kmeans.cluster_centers_
plt.scatter(centers[:,0], centers[:,1], c='black', s=200, alpha=0.5, marker='x')
for i, center in enumerate(centers):
    plt.annotate(f'Cluster {i}', (center[0], center[1]), textcoords="offset points", xytext=(0,10), ha='center')

plt.xlabel('Recency')
plt.ylabel('Frequency')
plt.legend()
plt.show()

As we can see after clustering, Cluster 0 & Cluster 3 are coinsiding with each other. Hence we might be lowering the number of clusters to 3.

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(X)
y_km = kmeans.predict(X)

# Plot the clusters
plt.figure(figsize=(8, 5))
plt.title('Customer Segmentation based on Recency and Frequency')
plt.scatter(X[:,0], X[:,1], c=y_km, s=50, cmap='Set1', label='Clusters')

# Plot and annotate the centers
centers = kmeans.cluster_centers_
plt.scatter(centers[:,0], centers[:,1], c='black', s=200, alpha=0.5, marker='x')
for i, center in enumerate(centers):
    plt.annotate(f'Cluster {i}', (center[0], center[1]), textcoords="offset points", xytext=(0,10), ha='center')

plt.xlabel('Recency')
plt.ylabel('Frequency')
plt.legend()
plt.show()

**Interpreting the Clusters with the help of RFM Group:**

Splitting the RFM values to 4 quantiles:

In [ ]:
quantiles = df_uk.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

def RScore(x,p,d):
  if x <= d[p][0.25]:
    return 1
  elif x <= d[p][0.5]:
    return 2
  elif x <= d[p][0.75]:
    return 3
  else:
    return 4
def FnMScore(x,p,d):
  if x <= d[p][0.25]:
    return 4
  elif x <= d[p][0.5]:
    return 3
  elif x <= d[p][0.75]:
    return 2
  else:
    return 1

In [ ]:
df_uk['R'] = df_uk['Recency'].apply(RScore, args=('Recency', quantiles, ))
df_uk['F'] = df_uk['Frequency'].apply(FnMScore, args=('Frequency', quantiles, ))
df_uk['M'] = df_uk['Monetary'].apply(FnMScore, args=('Monetary', quantiles, ))
df_uk.reset_index().head()

**Calculating RFM Group and RFM Score from the RFM segmentation:**

In [ ]:
# add RFM group column
df_uk['RFMGroup'] = df_uk['R'].map(str) + df_uk['F'].map(str) + df_uk['M'].map(str)

# calculate RFM score from RFM group column
df_uk['RFMScore'] = df_uk[['R', 'F', 'M']].sum(axis=1)
df_uk.reset_index().head()

In [ ]:
#finding the clusters from the dataset

df_uk['Cluster'] = kmeans.labels_
df_uk.head()

In [ ]:
sns.countplot(x = df_uk['Cluster'])

In [ ]:
# Grouping by cluster label and calculate mean
cluster_averages = df_uk.groupby('Cluster').mean()
cluster_averages

## **8. Interpretetion**

Cluster 0:

* Recency: High (average around 30 days)
* Frequency: Low (average around 1 transactions)
* Monetary: Low (average around $16)
* Interpretation: Customers in this group are likely to be 'At-Risk' or 'Lapssesd' customers. They haven't made any recent purchase and when they did they neither purchased anything frequently nor they spent much. Engaging them with reactivation campaigns or exploring why they haven't returned can be a strategic move.
But since the data is from an online gifting retail site, it can be derived that people dont buy gifts much frequently, unless they have any occasions to suffice the reaoning. Hence, if the former move to engage the customers doesn't work, seasonal offers and campaigns can be initiated for this segment of customers to have their loyalty to buy from this store whenever they have to buy gifts.


Cluster 1:

* Recency: High (average around 25 days)
* Frequency: High (average around 33 transactions)
* Monetary: High (average around $700)
* Interpretetion:  This cluster represents your 'Champions' or 'Loyal' customers. They shop frequently, recently, and spend the most. They are the most valuable segment, likely to respond positively to new offers, up-sell and cross-sell opportunities. This segment might consist of other third party retailers, so maintaining their high engagement level is crucial, and they can also be targeted for feedback or as brand ambassadors.

Cluster 2:

* Recency: Low (average around 4 days)
* Frequency: Moderate (average around 2 transactions)
* Monetary: Moderate (average around $13)
* Interpretetion: Customers in this cluster can be seen as 'Potential Loyalists' or 'Promising' customers. They have a balanced score in all three RFM metrics. These customers have the potential to become more valuable if properly engaged. Tailored marketing strategies, loyalty programs, and incentives to increase their purchase frequency and value can be effective.



# **Conclusion**

* The above above clustering is done with RFM data using K-Means Clustering as all 3 will provide the best information for the business requirement.

* Cluster 0 has high Recency, low frequency and monetary value thus depicting the most risked class of the entire sample consisting of above 13k customers.
* Cluster 1 has high values in all recency, frequency and monetary value depicting about 1000+ golden customers.
* Cluster 2 has low recency and moderaate frequency and monetary values consisting of about 2500+ loyal customers.